In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle as pk
import random
import sys
from sklearn.model_selection import train_test_split
from random import shuffle
import matplotlib.pyplot as plt
import scipy.stats as stats
import math

In [2]:
dataset = pd.read_feather('./normalized_one_hot_encoded_data_Charlie.ftr')
# normalized_data = pd.read_pickle('./normalized_one_hot_encoded_data.pkl')

In [3]:
dataset.shape

(2182969, 484)

In [4]:
data = dataset.sample(frac=0.80, random_state=786).reset_index(drop=True)
data_tune_and_unseen = dataset.drop(data.index).reset_index(drop=True)
data_tune = data_tune_and_unseen.sample(frac=0.75, random_state=786).reset_index(drop=True)
data_unseen = data_tune_and_unseen.drop(data_tune.index).reset_index(drop=True)

data_train = data.copy()
data_validation = data_tune.copy()
data_test = data_unseen.copy()
data_train.drop(labels=["TRANSACTIONCANCELLED"],axis=1,inplace=True)
data_test.drop(labels=["TRANSACTIONCANCELLED"],axis=1,inplace=True)
data_train_true = data[["TRANSACTIONCANCELLED"]]
data_test_true = data_unseen[["TRANSACTIONCANCELLED"]]

print('Data for Modeling: ' + str(data_train.shape))
print('Unseen Data For Predictions: ' + str(data_test.shape))

Data for Modeling: (1746375, 483)
Unseen Data For Predictions: (109148, 483)


In [5]:
data_test

,FUND_TYPE_0,FUND_TYPE_1,FUND_TYPE_2,FUND_TYPE_3,FUND_TYPE_4,FUND_TYPE_5,FUND_TYPE_6,FUND_TYPE_7,FUND_TYPE_8,FUND_TYPE_9,...,days_223,days_224,days_225,days_226,days_227,days_228,AMOUNT,INTEREST_VALUE,QUANTITY,COST
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.299801,0.499998,0.496066,0.43047
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.299801,0.500000,0.496066,0.43047
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.299802,0.500000,0.496066,0.43047
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.299802,0.500001,0.496066,0.43047
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.299801,0.499989,0.496050,0.43047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109143,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.299801,0.500000,0.496066,0.43047
109144,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.299801,0.500000,0.496127,0.43047
109145,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.299801,0.500000,0.496005,0.43047
109146,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.299801,0.500000,0.496005,0.43047


In [6]:
data_train_true

,TRANSACTIONCANCELLED
0,0
1,0
2,0
3,0
4,0
...,...
1746370,0
1746371,0
1746372,0
1746373,0


In [7]:
def evaluation_metrics(df):
    true_positive = df[(df["Anomaly"]==1) & (df["TRANSACTIONCANCELLED"]==1)]
    false_positive = df[(df["Anomaly"]==1) & (df["TRANSACTIONCANCELLED"]==0)]
    false_negative = df[(df["Anomaly"]==0) & (df["TRANSACTIONCANCELLED"]==1)]
    true_negative = df[(df["Anomaly"]==0) & (df["TRANSACTIONCANCELLED"]==0)]
    tp = len(true_positive)
    fp = len(false_positive)
    fn = len(false_negative)
    tn = len(true_negative)
    print(tp+fp+fn+tn)
    accuracy = (tp+tn)/(tp+fp+fn+tn)
    precision = tp/(tp+fp)
    recall = tp/(tp+tn)
    f1 = 2*precision*recall/(precision+recall)
    return accuracy, precision, recall, f1    

In [8]:
from pycaret.anomaly import *

In [24]:
exp_ano101 = setup(data_train, normalize = True, session_id = 123)

,Description,Value
0,session_id,123
1,Original Data,"(1746375, 483)"
2,Missing Values,False
3,Numeric Features,483
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1746375, 483)"
9,CPU Jobs,-1


In [25]:
pca = create_model(model='pca', fraction=0.1)

In [23]:
print(tuned_pca)

PCA(contamination=0.1, copy=True, iterated_power='auto', n_components=None,
  n_selected_components=None, random_state=123, standardization=True,
  svd_solver='auto', tol=0.0, weighted=True, whiten=False)


In [26]:
print(pca)

PCA(contamination=0.1, copy=True, iterated_power='auto', n_components=None,
  n_selected_components=None, random_state=123, standardization=True,
  svd_solver='auto', tol=0.0, weighted=True, whiten=False)


In [27]:
pca_results = assign_model(pca)
pca_results

,FUND_TYPE_0,FUND_TYPE_1,FUND_TYPE_2,FUND_TYPE_3,FUND_TYPE_4,FUND_TYPE_5,FUND_TYPE_6,FUND_TYPE_7,FUND_TYPE_8,FUND_TYPE_9,...,days_225,days_226,days_227,days_228,AMOUNT,INTEREST_VALUE,QUANTITY,COST,Anomaly,Anomaly_Score
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.430470,0,8.267696e+17
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.430470,0,1.570717e+18
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.430470,0,1.192970e+18
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299775,0.499457,0.496031,0.430470,0,1.094900e+18
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299795,0.499874,0.496031,0.430470,0,1.153116e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746370,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.430470,0,9.268569e+17
1746371,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.430470,0,1.640843e+18
1746372,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.499987,0.496064,0.430422,0,1.557737e+18
1746373,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496081,0.430556,1,3.027439e+18


In [28]:
unseen_predictions = predict_model(pca, data=data_test)
unseen_predictions

,FUND_TYPE_0,FUND_TYPE_1,FUND_TYPE_2,FUND_TYPE_3,FUND_TYPE_4,FUND_TYPE_5,FUND_TYPE_6,FUND_TYPE_7,FUND_TYPE_8,FUND_TYPE_9,...,days_225,days_226,days_227,days_228,AMOUNT,INTEREST_VALUE,QUANTITY,COST,Anomaly,Anomaly_Score
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.499998,0.496066,0.43047,0,2.161575e+18
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.500000,0.496066,0.43047,0,1.177124e+18
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299802,0.500000,0.496066,0.43047,0,1.178992e+18
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299802,0.500001,0.496066,0.43047,0,1.172010e+18
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.299801,0.499989,0.496050,0.43047,0,2.161575e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109143,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0.299801,0.500000,0.496066,0.43047,1,8.858708e+19
109144,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.299801,0.500000,0.496127,0.43047,1,8.858794e+19
109145,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.299801,0.500000,0.496005,0.43047,1,8.858794e+19
109146,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0.299801,0.500000,0.496005,0.43047,1,8.859268e+19


In [29]:
test_results = pd.concat([unseen_predictions, data_test_true], axis=1)
test_results

,FUND_TYPE_0,FUND_TYPE_1,FUND_TYPE_2,FUND_TYPE_3,FUND_TYPE_4,FUND_TYPE_5,FUND_TYPE_6,FUND_TYPE_7,FUND_TYPE_8,FUND_TYPE_9,...,days_226,days_227,days_228,AMOUNT,INTEREST_VALUE,QUANTITY,COST,Anomaly,Anomaly_Score,TRANSACTIONCANCELLED
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.299801,0.499998,0.496066,0.43047,0,2.161575e+18,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.299801,0.500000,0.496066,0.43047,0,1.177124e+18,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.299802,0.500000,0.496066,0.43047,0,1.178992e+18,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.299802,0.500001,0.496066,0.43047,0,1.172010e+18,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.299801,0.499989,0.496050,0.43047,0,2.161575e+18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109143,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0.299801,0.500000,0.496066,0.43047,1,8.858708e+19,0
109144,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0.299801,0.500000,0.496127,0.43047,1,8.858794e+19,0
109145,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0.299801,0.500000,0.496005,0.43047,1,8.858794e+19,0
109146,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0.299801,0.500000,0.496005,0.43047,1,8.859268e+19,0


In [30]:
len(test_results["Anomaly"])

109148

In [31]:
accuracy, precision, recall, f1 = evaluation_metrics(test_results)

109148


In [32]:
accuracy, precision, recall, f1

(0.8662549932202147,
 0.02738606172082567,
 0.004251718667371761,
 0.007360682602605535)

In [18]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [22]:
pca_tune = setup(data_validation, normalize = True, session_id = 123)
tuned_pca = tune_model(model="pca", supervised_target = "TRANSACTIONCANCELLED")

ValueError: Function <function tune_model_unsupervised.<locals>.highlight_min at 0x7f58ec504940> resulted in the apply method collapsing to a Series.
Usually, this is the result of the function returning a single value, instead of list-like.

In [20]:
unseen_predictions = predict_model(tuned_pca, data=data_test)
test_results = pd.concat([unseen_predictions, data_test_true], axis=1)
accuracy, precision, recall, f1 = evaluation_metrics(test_results)
accuracy, precision, recall, f1

KeyError: "['TRANSACTIONCANCELLED'] not in index"

### Clustering-Based Local Outlier

In [9]:
cluster_env = setup(data_train, normalize = True, session_id = 101)
cluster = create_model('cluster')
cluster_result = assign_model(cluster)
unseen_predictions = predict_model(cluster, data=data_test)
test_results = pd.concat([unseen_predictions, data_test_true], axis=1)
accuracy, precision, recall, f1 = evaluation_metrics(test_results)
accuracy, precision, recall, f1

109148


(0.9065763916883498,
 0.015842191332788226,
 0.0015664318703196532,
 0.0028509679496022443)

### Histogram Outlier Detector

In [15]:
histogram_env = setup(data_train, normalize = True, session_id = 101)
histogram = create_model(model = 'histogram', fraction = 0.99)
histogram_result = assign_model(histogram)
unseen_predictions = predict_model(histogram, data=data_test)
test_results = pd.concat([unseen_predictions, data_test_true], axis=1)
accuracy, precision, recall, f1 = evaluation_metrics(test_results)
accuracy, precision, recall, f1

109148


(0.024837834866419908,
 0.006746920492721164,
 0.266691257838436,
 0.013160888678541196)

### Stochastic Outlier Selection

In [ ]:
sos_env = setup(data_train, session_id = 101)
sos = create_model(model = 'sos')
sos_result = assign_model(sos)
unseen_predictions = predict_model(sos, data=data_test)
test_results = pd.concat([unseen_predictions, data_test_true], axis=1)
accuracy, precision, recall, f1 = evaluation_metrics(test_results)
accuracy, precision, recall, f1

,Description,Value
0,session_id,101
1,Original Data,"(1746375, 483)"
2,Missing Values,False
3,Numeric Features,483
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(1746375, 483)"
9,CPU Jobs,-1


IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:48:11
Status,. . . . . . . . . . . . . . . . . .,Fitting 0.05 Fraction
Estimator,. . . . . . . . . . . . . . . . . .,Stochastic Outlier Selection
